# Compute Optimal Generalization for Multistep Problems
All the shared functions are in the xcslib python library.

In [1]:
import os
import pandas as pd
import numpy as np
import pathlib
from pathlib import Path

# we apply AgglomerativeClustering with a limit of 2*EpsilonError 
# so that it does not merge whose distance is above that value 
# DBSCAN might also be used setting adequate density parameters
from sklearn.cluster import AgglomerativeClustering

In [2]:
from xcslib2 import load_xcslib_population
from xcslib2 import compute_cluster_prediction_statistics
from xcslib2 import generate_espresso_for_action,generate_complete_representation,load_pla

## Overwrites some of the functions

In [3]:
def cluster_expected_payoff(population, error_threshold=10, min_sample=5, no_clusters=None, use_numerosity=True, use_dbscan=False, verbose=False):

    if verbose:
        print("POPULATION SIZE = %d"%len(population))
    
    if use_dbscan:
        if verbose:
            print("WARNING: using DBSCAN (old version).")
        clustering = DBSCAN(eps=error_threshold, min_samples=min_sample)
    else:
        clustering = AgglomerativeClustering(n_clusters=no_clusters,distance_threshold=2*error_threshold)
        
    predictions = generate_prediction_vector(population, use_numerosity=use_numerosity)

    if verbose:
        print("POPULATION SIZE = %d PREDICTIONS TO CLUSTER = %d"%(len(population),len(predictions)))
    
    clustering.fit(predictions)
          
    population['label'] = clustering.labels_

    if verbose:
        print("# LABELS = %d # LABEL VALUES = %d"%(len(clustering.labels_),len(population['label'].unique())))
        print(population[['label','prediction']].groupby(by=['label']).mean())
        print(population[['label','prediction']].groupby(by=['label']).std())

    return population

def generate_prediction_vector(population, use_numerosity=True):
    
    if (not use_numerosity):
        return population.values.reshape(-1,1)
    
    predictions = []
    
    for i,row in population.iterrows():
        predictions += [row['prediction']]*row['numerosity']

    return np.array(predictions).reshape(-1,1)

In [27]:
def generate_optimized_population(problem, error_threshold, source_directory, destination_directory, espresso_command, experiment=0, create_complete_representation=True,run_minimization=True, verbose=False, no_clusters=None):
    """run_minimization: if true it also calls the espresso functions."""
    
    # create the destination directory
    pathlib.Path(destination_directory).mkdir(parents=True,exist_ok=True)

    # XCS configuration file
    config_path = source_directory + "/confsys.%s"%(problem)
    
    # population file
    population_path = "%s/population.%s-%04d.gz"%(source_directory,problem,experiment)

    # path to pla files
    pla_path = "%s/%s.pla"%(destination_directory,problem)

    prediction_statistics_path = "%s/%s_ps.csv"%(destination_directory,problem)    
    
    ################################################################################
    # Step 1. Load the population
    #         Cluster the payoff values
    #         Compute the statistics of the clusters
    ################################################################################
    
    if (verbose):
        print("Step 1. Load the population")
        print("        Cluster the payoff values")
        print("        Compute the statistics of the clusters")
        print("\n\n")
        
    # load population
    df_population = load_xcslib_population(population_path)

    # save the population as csv
    df_population.to_csv(population_path.replace(".gz",".csv.gz"),index=False,compression="gzip")

    # cluster the payoff values
    df_cluster = cluster_expected_payoff(df_population, error_threshold=error_threshold, min_sample=1, no_clusters=no_clusters)

    # compute the stastistics for every values
    prediction_statistics, df_prediction_statistics = compute_cluster_prediction_statistics(df_cluster)
    df_prediction_statistics.to_csv(prediction_statistics_path,index=False)

    # payoff dictionary
    payoff_values_table = df_cluster[['label','prediction']].groupby(by=['label']).mean()    
    payoff_values_table.reset_index(inplace=True)

    payoff_values = {}
    for i,row in payoff_values_table.iterrows():
        payoff_values[row['label']]=row['prediction']
    
    # compute population size
    with open(config_path,"r") as CONFIG:
        str_config = CONFIG.read()
    population_size = int([s.strip() for s in str_config.split("\n") if (s.strip()!="" and s.strip()[:17]=='population size =')][0].split(" = ")[1])        
    no_action_bits = int([s.strip() for s in str_config.split("\n") if (s.strip()!="" and  s.strip()[:17]=='number of bits = ')][0].split(" = ")[1])    

    ################################################################################
    # Step 2. Create the pla file for each action
    ################################################################################

    if (verbose):
        print("Step 2. Create the pla file for each action")
        print("\n\n")
    
    actions = df_population['action'].unique()
    actions.sort()
    
    espresso_files = {}
    for action in actions:
        espresso_action = generate_espresso_for_action(df_cluster,action)
        espresso_files[str(action)] = espresso_action

    action_pla_paths = {}

    for action in espresso_files.keys():    

        action_pla_path = pla_path.replace(".pla","_a"+action+".pla")

        action_pla_paths[action] = action_pla_path

        with open(action_pla_path, "w") as OUTPUT:
            OUTPUT.write(espresso_files[action])
            
    ################################################################################
    # Step 3. Generate complete representation for every action
    ################################################################################

    # generate the filenames for the complete and minimized representations so to 
    # check whether it is necessary to create them or they are already available 
    # so we skip the step

    complete_action_pla_paths = {}
    minimized_pla_paths = {}

    for action in action_pla_paths.keys():
        action_pla_path = action_pla_paths[action]
        complete_action_pla_paths[action] = action_pla_path.replace(".pla","_complete.pla")    
        minimized_pla_paths[action] = complete_action_pla_paths[action].replace(".pla","_minimized.pla")

    # if the creation of the complete representation is needed
    if (create_complete_representation):

        if (verbose):
            print("Step 3. Generate complete representation for every action")
            print("\n\n")
        
        # complete_action_pla_paths = {}
        for action in action_pla_paths.keys():
            if (verbose):
                print("        Action ",str(action))
            
            generate_complete_representation(action_pla_paths[action],complete_action_pla_paths[action])
                    
    ################################################################################
    # Step 4. Optimize the PLA representation for each action
    ################################################################################

    if (not run_minimization):
        print("The parameter run_minimization is set to False\nthus the process stops at generation")
        return
        
    if (verbose):
        print("Step 4. Optimize the PLA representation for each action")
        print("\n\n")
    
    # minimized_pla_paths = {}
    for action in complete_action_pla_paths.keys():

        # action_pla_path = complete_action_pla_paths[action]

        # minimized_pla_path = action_pla_path.replace(".pla","_minimized.pla")

        # minimized_pla_paths[action] = minimized_pla_path

        if (not Path(minimized_pla_paths[action]).is_file()):
            print("PATH => ",minimized_pla_paths[action])
            minimization_command = "%s %s > %s"%(espresso_command,complete_action_pla_paths[action], minimized_pla_paths[action])

            minimization_result = os.system(minimization_command)
            if (minimization_result!=0):
                print("MINIMIZATION FAILED FOR ACTION "+action)
                print(minimization_command)      
        else:
            if (verbose):
                print("       ==> skipping minimization since the minimized file already exists.")
                print("           %s"%(minimized_pla_paths[action].split(os.sep)[-1]))

    ################################################################################
    # Step 5. Generate the population by joining all the minimized solutions
    ################################################################################

    if (verbose):
        print("Step 5. Generate the population by joining all the minimized solutions")
        print("\n\n")
            
    pla = {}
    for action in minimized_pla_paths.keys():
        print("==>",minimized_pla_paths[action])
        pla[action] = load_pla(minimized_pla_paths[action])
        
    population = {
        'id':[],
        'condition':[],
        'action':[],
        'prediction':[],
        'error':[],
        'fitness':[],
        'action_set_size':[],
        'experience':[],
        'numerosity':[],
    }

    current_id = 0

    for action in pla.keys():
        input_output, no_input_bits, no_output_bits, input_labels, output_labels = pla[action]

        for input_string in input_output.keys():
            output_string = input_output[input_string]

            condition = input_string.replace("-","#")        
            prediction = payoff_values[output_string.index('1')]       

            population['id'].append(current_id)
            population['condition'].append(condition)
            population['action'].append(action)
            population['prediction'].append(prediction)
            population['error'].append(0.0)
            population['fitness'].append(1.0)
            population['action_set_size'].append(100)
            population['experience'].append(100)
            population['numerosity'].append(0)

            current_id = current_id + 1

    df = pd.DataFrame.from_dict(population)
    
    numerosity = int(float(population_size)/len(df))
    numerosity_values = [numerosity]*(len(df)-1)
    numerosity_values.append(population_size-sum(numerosity_values))
    df['numerosity'] = numerosity_values    
    
    minimized_population_path = source_directory+"/population.%s-%04d.gz"%(problem+"_min",experiment)

    df.to_csv(minimized_population_path,sep="\t", compression="gzip",index=False,header=False)
    
    df[['condition','action','prediction']].to_csv(source_directory+"/optimal_population.%s-%04d"%(problem,experiment),sep="\t", index=False,header=False)
    

In [28]:
def generate_population_from_pla_files(minimized_pla_paths, population_size, problem, payoff_values, destination_directory):

    original_pla = {}    
    pla = {}
    for minimized_pla_path in minimized_pla_paths:
        action = minimized_pla_path.split(os.sep)[-1].split("_")[1][1:]
        pla[action] = load_pla(minimized_pla_path)
        
    population = {
        'id':[],
        'condition':[],
        'action':[],
        'prediction':[],
        'error':[],
        'fitness':[],
        'action_set_size':[],
        'experience':[],
        'numerosity':[],
    }

    current_id = 0

    for action in pla.keys():
        input_output, no_input_bits, no_output_bits, input_labels, output_labels = pla[action]

        # print("OUTPUT LABELS ",output_labels)
        
        output_labels_list = [int(label[1:]) for label in output_labels[4:].strip().split(" ")]
        # print(output_labels_list)

        for input_string in input_output.keys():
            output_string = input_output[input_string]

            condition = input_string.replace("-","#")        
            prediction = payoff_values[output_string.index('1')]       

            population['id'].append(current_id)
            population['condition'].append(condition)
            population['action'].append(action)
            population['prediction'].append(prediction)
            population['error'].append(0.0)
            population['fitness'].append(1.0)
            population['action_set_size'].append(100)
            population['experience'].append(100)
            population['numerosity'].append(0)

            current_id = current_id + 1

    df = pd.DataFrame.from_dict(population)
    
    numerosity = int(float(population_size)/len(df))
    numerosity_values = [numerosity]*(len(df)-1)
    numerosity_values.append(population_size-sum(numerosity_values))
    df['numerosity'] = numerosity_values    
    
    experiment = 0
    minimized_population_path = destination_directory+"/population.%s-%04d.gz"%(problem+"_min",experiment)

    df.to_csv(minimized_population_path,sep="\t", compression="gzip",index=False,header=False)
    
    df[['condition','action','prediction']].to_csv(destination_directory+"/optimal_population.%s-%04d"%(problem,experiment),sep="\t", index=False,header=False)
 

## Settings

In [23]:
# espresso_command = "../../../espresso-logic-master/bin/espresso -Dexact"
espresso_command = "../espresso-logic-master/bin/espresso "
experiments_directory = "../experiments-qlearning/"

## Experimenti for Woods1q

In [ ]:
# directory where the files are saved
problem_directory = "woods1"

# extension used to save the q-table
problem_extension = "woods1q"

source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)

# error threshold for XCS definition of accurate classifiers
error_threshold = 10
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command, verbose=True)

Step 1. Load the population
        Cluster the payoff values
        Compute the statistics of the clusters



Step 2. Create the pla file for each action



Step 3. Generate complete representation for every action



        Action  000
        Action  001
        Action  010
        Action  011
        Action  100
        Action  101
        Action  110
        Action  111
Step 4. Optimize the PLA representation for each action



       ==> skipping minimization since the minimized file already exists.
           woods1q_a000_complete_minimized.pla
       ==> skipping minimization since the minimized file already exists.
           woods1q_a001_complete_minimized.pla
       ==> skipping minimization since the minimized file already exists.
           woods1q_a010_complete_minimized.pla
       ==> skipping minimization since the minimized file already exists.
           woods1q_a011_complete_minimized.pla
       ==> skipping minimization since the minimized file already exists.
   

## Experimenti for Maze4

In [ ]:
problem_directory = "maze4"
problem_extension = "maze4q"
source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)
error_threshold = 10
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command)

## Experimenti for Maze5

In [ ]:
problem_directory = "maze5"
problem_extension = "maze5q"
source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)
error_threshold = 10
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command)


## Experimenti for Maze6

In [ ]:
%%time
problem_directory = "maze6"
problem_extension = "maze6q"
source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)
error_threshold = 10
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command)

CPU times: user 7.02 s, sys: 61.8 ms, total: 7.08 s
Wall time: 7.11 s


## Experiments for Woods14

In [ ]:
%%time
problem_directory = "woods14"
problem_extension = "woods14q"
source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)
error_threshold = 0.1
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command)

CPU times: user 3.72 s, sys: 45.8 ms, total: 3.76 s
Wall time: 3.77 s


## Experimenti for Woods2
The full run takes around 35 minutes - 1.47 minutes of actual time. The bottleneck is the creation of the complete representations. In this case, we turned it off since the minimized pla files are already available. 

In [26]:
%%time
problem_directory = "woods2"
problem_extension = "woods2q"
source_directory = "%s/%s/"%(experiments_directory,problem_directory)
destination_directory = "%s/%s/pla/"%(experiments_directory,problem_directory)
error_threshold = 10
generate_optimized_population(problem_extension, error_threshold, source_directory, destination_directory, espresso_command, \
    create_complete_representation=False)

==> ../experiments-qlearning//woods2/pla//woods2q_a000_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a001_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a010_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a011_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a100_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a101_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a110_complete_minimized.pla
==> ../experiments-qlearning//woods2/pla//woods2q_a111_complete_minimized.pla
    id                 condition action  prediction  error  fitness  \
0    0  ###################1####    000       700.0    0.0      1.0   
1    1  ###################0####    000       490.0    0.0      1.0   
2    2  ########################    001       490.0    0.0      1.0   
3    3  #############1##########    010       700.0    0.0      1.0   
4    4  #############